In [1]:
#read the data 
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

#import data set
import json
import gzip

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.dumps(eval(l))

f = open("output.strict", 'w')
for l in parse("Downloads/reviews_Automotive_5.json.gz"):
  f.write(l + '\n')

#read into pandas data frame
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Downloads/reviews_Automotive_5.json.gz')

#read image features
import array

def readImageFeatures(path):
  f = open(path, 'rb')
  while True:
    asin = f.read(10)
    if asin == '': break
    a = array.array('f')
    a.fromfile(f, 4096)
    yield asin, a.tolist()


In [2]:
#sum ratings
ratings = []

for review in parse("Downloads/reviews_Automotive_5.json.gz"):
  ratings.append(review['overall'])

print(sum(ratings) / len(ratings))

df

#set refined df
review_df = df[['reviewerID','reviewText','overall']]

#create id index
review_df = df.set_index(['reviewerID'])

review_df

# Define a function to clean the text
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    import re
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text



4.471840961266058


In [3]:
# Cleaning the text in the review column 
review_df['reviewText'] = review_df['reviewText'].apply(clean)
review_df

#remove neutral reviews
review_df = review_df[review_df['overall'] != 2.0]
review_df = review_df[review_df['overall'] != 3.0]
review_df = review_df[review_df['overall'] != 4.0]
print(review_df.shape)
review_df.head(5)

review_df["overall"].value_counts()

#identify all text items
tweet = review_df.reviewText.values


(14470, 8)


In [ ]:
#breakdown text into tensors
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)


In [1]:
#replace words with assigned numbers
encoded_docs = tokenizer.texts_to_sequences(tweet)


NameError: name 'tokenizer' is not defined

In [ ]:
#pad sentences to create equal length
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen=200)


In [ ]:
#run tensorflow to avoid overfitting model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
vocab_size = 10000
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

#train sentiment analysis model
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=5, batch_size=32)


In [1]:
#train sentiment analysis model
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=5, batch_size=32)



NameError: name 'model' is not defined

In [2]:
#plot training accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()
plt.savefig("Accuracy plot.jpg")


NameError: name 'history' is not defined

In [3]:
#plot loss metric
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.savefig("Loss plt.jpg")


NameError: name 'history' is not defined

In [4]:
#make predictions
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    print("Predicted label: ", sentiment_label[1][prediction])
test_sentence1 = "I enjoyed this product."
predict_sentiment(test_sentence1)
test_sentence2 = "This is the worst product experience of my life!"
predict_sentiment(test_sentence2)


NameError: name 'tokenizer' is not defined